![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20DistilBertForSequenceClassification.ipynb)

## Import DistilBertForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.3.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DistilBERT models trained/fine-tuned for token classification via `DistilBertForSequenceClassification` or `TFDistilBertForSequenceClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDistilBertForSequenceClassification](https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforsequenceclassification)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) model from HuggingFace as an example
- In addition to `TFDistilBertForSequenceClassification` we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer 
import tensorflow as tf

MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")       
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 523352
-rw-r--r--  1 maziyar  staff        735 Dec 15 16:45 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 15 16:45 saved_model
-rw-r--r--  1 maziyar  staff  267951808 Dec 15 16:45 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 10000
drwxr-xr-x  2 maziyar  staff       64 Dec 15 16:45 assets
-rw-r--r--  1 maziyar  staff       53 Dec 15 16:45 fingerprint.pb
-rw-r--r--  1 maziyar  staff    80253 Dec 15 16:45 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  5032374 Dec 15 16:45 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 15 16:45 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 472
-rw-r--r--  1 maziyar  staff     125 Dec 15 16:45 special_tokens_map.json
-rw-r--r--  1 maziyar  staff     436 Dec 15 16:45 tokenizer_config.json
-rw-r--r--  1 maziyar  staff  231508 Dec 15 16:45 vocab.txt


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.txt` from the tokenizer
- All we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}

In [ ]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 464
-rw-r--r--  1 maziyar  staff      17 Dec 15 16:46 labels.txt
-rw-r--r--  1 maziyar  staff  231508 Dec 15 16:46 vocab.txt


## Import and Save DistilBertForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.4
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.4


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DistilBertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

sequenceClassifier = DistilBertForSequenceClassification.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DistilBertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 532864
-rw-r--r--  1 maziyar  staff  272823081 Dec 15 16:46 distilbert_classification_tensorflow
drwxr-xr-x  5 maziyar  staff        160 Dec 15 16:46 fields
drwxr-xr-x  6 maziyar  staff        192 Dec 15 16:46 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForSequenceClassification model 😊 

In [ ]:
sequenceClassifier_loaded = DistilBertForSequenceClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

That's it! You can now go wild and use hundreds of `DistilBertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀 


You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
sequenceClassifier_loaded.getClasses()

['POSITIVE', 'NEGATIVE']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier_loaded    
])

# couple of simple examples
example = spark.createDataFrame([["I love you!"], ['I feel lucky to be here.'], ['I hate her!']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

+--------------------+----------+
|                text|    result|
+--------------------+----------+
|         I love you!|[POSITIVE]|
|I feel lucky to b...|[POSITIVE]|
|         I hate her!|[NEGATIVE]|
+--------------------+----------+



That's it! You can now go wild and use hundreds of `DistilBertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀 